In [5]:
'''http://backtrader.readthedocs.org/en/latest/quickstart/quickstart.html'''
sudo apt-get install build-essential python-dev
pip install bt # backtest
pip install backtrader

SyntaxError: invalid syntax (<ipython-input-5-4f17c1d0ffdd>, line 2)

In [1]:
from __future__ import (absolute_import, division, print_function,
                        unicode_literals)

import datetime  # For datetime objects
import os.path  # To manage paths
import sys  # To find out the script name (in argv[0])

# Import the backtrader platform
import backtrader as bt


# Create a Stratey
class TestStrategy(bt.Strategy):
    params = (
        ('maperiod', 15),
        ('stake', 10),
    )

    def log(self, txt, dt=None):
        ''' Logging function fot this strategy'''
        dt = dt or self.datas[0].datetime.date(0)
        print('%s, %s' % (dt.isoformat(), txt))

    def __init__(self):
        # Keep a reference to the "close" line in the data[0] dataseries
        self.dataclose = self.datas[0].close

        # Set the sizer stake from the params
        self.sizer.setsizing(self.params.stake)

        # To keep track of pending orders and buy price/commission
        self.order = None
        self.buyprice = None
        self.buycomm = None

        # Add a MovingAverageSimple indicator
        self.sma = bt.indicators.SimpleMovingAverage(
            self.datas[0], period=self.params.maperiod)

        # Indicators for the plotting show
        bt.indicators.ExponentialMovingAverage(self.datas[0], period=25)
        bt.indicators.WeightedMovingAverage(self.datas[0], period=25,
                                            subplot=True)
        bt.indicators.StochasticSlow(self.datas[0])
        bt.indicators.MACDHisto(self.datas[0])
        rsi = bt.indicators.RSI(self.datas[0])
        bt.indicators.SmoothedMovingAverage(rsi, period=10)
        bt.indicators.ATR(self.datas[0], plot=False)

    def notify_order(self, order):
        if order.status in [order.Submitted, order.Accepted]:
            # Buy/Sell order submitted/accepted to/by broker - Nothing to do
            return

        # Check if an order has been completed
        # Attention: broker could reject order if not enougth cash
        if order.status in [order.Completed, order.Canceled, order.Margin]:
            if order.isbuy():
                self.log(
                    'BUY EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                    (order.executed.price,
                     order.executed.value,
                     order.executed.comm))

                self.buyprice = order.executed.price
                self.buycomm = order.executed.comm
            else:  # Sell
                self.log('SELL EXECUTED, Price: %.2f, Cost: %.2f, Comm %.2f' %
                         (order.executed.price,
                          order.executed.value,
                          order.executed.comm))

            self.bar_executed = len(self)

        # Write down: no pending order
        self.order = None

    def notify_trade(self, trade):
        if not trade.isclosed:
            return

        self.log('OPERATION PROFIT, GROSS %.2f, NET %.2f' %
                 (trade.pnl, trade.pnlcomm))

    def next(self):
        # Simply log the closing price of the series from the reference
        self.log('Close, %.2f' % self.dataclose[0])

        # Check if an order is pending ... if yes, we cannot send a 2nd one
        if self.order:
            return

        # Check if we are in the market
        if not self.position:

            # Not yet ... we MIGHT BUY if ...
            if self.dataclose[0] > self.sma[0]:

                # BUY, BUY, BUY!!! (with all possible default parameters)
                self.log('BUY CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.buy()

        else:

            if self.dataclose[0] < self.sma[0]:
                # SELL, SELL, SELL!!! (with all possible default parameters)
                self.log('SELL CREATE, %.2f' % self.dataclose[0])

                # Keep track of the created order to avoid a 2nd order
                self.order = self.sell()



        
if __name__ == '__main__':
    cerebro = bt.Cerebro() # Create a cerebro entity
    cerebro.broker.setcash(100000.0) # portfolio cash
    # Set the commission - 0.1% ... divide by 100 to remove the %
    cerebro.broker.setcommission(commission=2.0, margin=2000.0, mult=10.0)
    cerebro.addstrategy(TestStrategy)     # Add a strategy

    # Get data
    data = bt.feeds.YahooFinanceCSVData(dataname='/home/andreyev/Downloads/table.csv')  # Create a Data Feed 
    cerebro.adddata(data)    # Add the Data Feed to Cerebro


    print('Starting Portfolio Value: %.2f' % cerebro.broker.getvalue())
    cerebro.run()
    print('Final Portfolio Value: %.2f' % cerebro.broker.getvalue())

Starting Portfolio Value: 100000.00
2015-09-21, Close, 114.72
2015-09-21, BUY CREATE, 114.72
2015-09-18, BUY EXECUTED, Price: 111.73, Cost: 20000.00, Comm 20.00
2015-09-18, Close, 112.97
2015-09-17, Close, 113.43
2015-09-16, Close, 115.91
2015-09-15, Close, 115.78
2015-09-14, Close, 114.82
2015-09-11, Close, 113.72
2015-09-10, Close, 112.09
2015-09-10, SELL CREATE, 112.09
2015-09-09, SELL EXECUTED, Price: 113.28, Cost: 20000.00, Comm 20.00
2015-09-09, OPERATION PROFIT, GROSS 155.00, NET 115.00
2015-09-09, Close, 109.68
2015-09-08, Close, 111.83
2015-09-04, Close, 108.80
2015-09-03, Close, 109.90
2015-09-02, Close, 111.86
2015-09-01, Close, 107.26
2015-08-31, Close, 112.28
2015-08-28, Close, 112.81
2015-08-28, BUY CREATE, 112.81
2015-08-27, BUY EXECUTED, Price: 111.75, Cost: 20000.00, Comm 20.00
2015-08-27, Close, 112.44
2015-08-26, Close, 109.22
2015-08-26, SELL CREATE, 109.22
2015-08-25, SELL EXECUTED, Price: 110.64, Cost: 20000.00, Comm 20.00
2015-08-25, OPERATION PROFIT, GROSS -111.

In [4]:
cerebro.plot()

/home/andreyev/anaconda3/lib/python3.4/site-packages/matplotlib/collections.py:590: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  if self._edgecolors == str('face'):
